Primero importamos las librerías que vamos a necesitar para nuestro modelo de regresión logística

In [2]:
import pandas as pd
import numpy as np

Ahora introducimos la ruta en la que están ubicados los archivos con la información que se analizará y tambíen aprovechamos para hacer un pequeño ajuste en la visualización del dataframe para meyor versatilidad.

In [ ]:
train = pd.read_csv('/home/luis-loreto/Documentos/Python/Titanic/Datos/train.csv')
test = pd.read_csv('/home/luis-loreto/Documentos/Python/Titanic/Datos/test.csv')

pd.options.display.max_columns = None
pd.options.display.max_rows = None

Iniciamos con el análisis exploratorio. Vemos cuántos registros tiene cada variable. Se puede observar que son 11 variables independientes y una dependiente: Survived. Ésta última será la variable para el aprendizaje en nuestro modelos de machine learning. 

In [15]:
Variables = {
    "Observaciones" : train.count(),
    "Missings": train.isnull().sum()
}
pd.DataFrame(Variables)

,Observaciones,Missings
PassengerId,891,0
Survived,891,0
Pclass,891,0
Name,891,0
Sex,891,0
Age,891,0
SibSp,891,0
Parch,891,0
Ticket,891,0
Fare,891,0


Vemos que las variables con missings son: Age, Cabin y Embarked. Debemos decidir si vamos a utilizar dichas variables y en su caso qué método de imputación elegir. Por el momento, descartaremos la variable Cabin, pues sus missings representan más de la mitad de las observaciones del dataframe. Usaremos algún método de imputación en las variables Age y Embarked para poder incluirlas en el modelo.

Ahora iniciaremos con la imputación para las variables Age y Embarked. Vamos a crear una función que le asigne valores aleatorios de su determinada variable a las observaciones con valores faltantes-

In [13]:
def hotdeck(data, variable):
    for column in train.columns:
        # Encontrar índices donde hay valores faltantes
        missings = data[data[f"{variable}"].isnull()].index
        
        for x in missings:
            # Seleccionar filas no faltantes
            no_missings = data[f"{variable}"].dropna()
            # Elegir un valor aleatorio de las filas no faltantes
            random_value = no_missings.sample(n=1).values[0]
            # Imputar el valor aleatorio en el índice correspondiente
            data.at[x, f"{variable}"] = random_value
            
    return data

In [14]:
train = hotdeck(train, "Age")
train = hotdeck(train, "Embarked")

Ahora vamos a corroborar que nuestro dataframe no tenga valores faltantes.

In [16]:
Variables = {
    "Observaciones" : train.count(),
    "Missings": train.isnull().sum()
}
pd.DataFrame(Variables)

,Observaciones,Missings
PassengerId,891,0
Survived,891,0
Pclass,891,0
Name,891,0
Sex,891,0
Age,891,0
SibSp,891,0
Parch,891,0
Ticket,891,0
Fare,891,0


Ahora iniciaremos con el proceso de One Hot Encoding. Crearemos variables dummies para aquellas con observaciones categóricas, es decir, para Sex, Pclass y Embarked

In [17]:
train = pd.get_dummies(train, columns = ['Sex', 'Pclass', 'Embarked'], drop_first=True, dtype=float)

In [18]:
train.head(3)

,PassengerId,Survived,Name,Age,SibSp,Parch,Ticket,Fare,Cabin,Sex_male,Pclass_2,Pclass_3,Embarked_Q,Embarked_S
0,1,0,"Braund, Mr. Owen Harris",22.0,1,0,A/5 21171,7.2500,NaN,1.0,0.0,1.0,0.0,1.0
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",38.0,1,0,PC 17599,71.2833,C85,0.0,0.0,0.0,0.0,0.0
2,3,1,"Heikkinen, Miss. Laina",26.0,0,0,STON/O2. 3101282,7.9250,NaN,0.0,0.0,1.0,0.0,1.0


Ahora creamos un conjunto de las variables independientes que usaremos y que ya pasaron por un proceso de one hot encoding y otro para la variable dependiente

In [19]:
X_train = train[["Age", "SibSp", "Parch", "Fare", "Sex_male", "Pclass_2", "Pclass_3", "Embarked_Q", "Embarked_S"]]
y_train = train[["Survived"]]

Independientemente del modelo que vayamos a usar, debemos darle el mismo tratamiento al conjunto de tratamiento. Primero identificaremos si tiene valores faltantes:

In [20]:
v_test = {
    "Observaciones" : test.count(),
    "Missings": test.isnull().sum()
}
pd.DataFrame(v_test)

,Observaciones,Missings
PassengerId,418,0
Pclass,418,0
Name,418,0
Sex,418,0
Age,332,86
SibSp,418,0
Parch,418,0
Ticket,418,0
Fare,417,1
Cabin,91,327


Vemos que sí, entonces ahora le aplicamos la función que creamos para la imputación. En este caso, se usará para las variables Fare y Age. Cabin se descarta porque los valores faltantes correspondonde a más de la mitad del total.

In [24]:
test = hotdeck(test, "Age")
test = hotdeck(test, "Fare")

In [25]:
v_test = {
    "Observaciones" : test.count(),
    "Missings": test.isnull().sum()
}
pd.DataFrame(v_test)

,Observaciones,Missings
PassengerId,418,0
Pclass,418,0
Name,418,0
Sex,418,0
Age,418,0
SibSp,418,0
Parch,418,0
Ticket,418,0
Fare,418,0
Cabin,91,327


Ahora crearemos nuestro conjunto para la variable dependiente y para la variable independiente, que se encuentra en otro archivo llamado "gender_submission"

Ahora aplicamos el one hot encodging

In [28]:
test = pd.get_dummies(test, columns = ['Sex', 'Pclass', 'Embarked'], drop_first=True, dtype=float)

In [33]:
X_test = test[["Age", "SibSp", "Parch", "Fare", "Sex_male", "Pclass_2", "Pclass_3", "Embarked_Q", "Embarked_S"]]
y_test = pd.read_csv('/home/luis-loreto/Documentos/Python/Titanic/Datos/gender_submission.csv')

In [30]:
y_test = pd.read_csv('/home/luis-loreto/Documentos/Python/Titanic/Datos/gender_submission.csv')

Ahora vamos a guardar los archivos ya preparados en nuestra carpeta "Datos"

In [34]:
X_train.to_csv('/home/luis-loreto/Documentos/Python/Titanic/Datos/X_train.csv', index=False)
y_train.to_csv('/home/luis-loreto/Documentos/Python/Titanic/Datos/y_train.csv', index=False)
X_test.to_csv('/home/luis-loreto/Documentos/Python/Titanic/Datos/X_test.csv', index=False)
y_test.to_csv('/home/luis-loreto/Documentos/Python/Titanic/Datos/y_test.csv', index=False)